In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [16]:
def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 200)

    def region_of_interest(frame):
        height = frame.shape[0]
        polygon = np.array([[(100, height), (900, height), (550, 250)]], dtype=np.int32)
        mask = np.zeros_like(frame)
        cv2.fillPoly(mask, polygon, 255)
        return cv2.bitwise_and(frame, mask)

    roi_image = region_of_interest(edges)

    lines = cv2.HoughLinesP(
        roi_image,
        rho=2,
        theta=np.pi/180,
        threshold=50,  
        minLineLength=20,  
        maxLineGap=10      
    )


    def draw_lines(img, lines):
        line_image = np.zeros_like(img)
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(line_image, (x1, y1), (x2, y2), (0, 0, 155), 10)
        return cv2.addWeighted(img, 0.9, line_image, 1, 1)

    return draw_lines(frame, lines)


input_video_path = r"C:\Projects\Lane_Detection\80400-572395752_tiny.mp4"
cap = cv2.VideoCapture(input_video_path)

while cap.isOpened():
    r, frame = cap.read()
    if r:
        processed_frame = process_frame(frame)
        cv2.imshow("Lane Detection", processed_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()
